# GT Saúde

O Grupo de Trabalho de Saúde definiu os indicadores sem a utilização dos formulários, em reuniões posteriores. Por isso, os indicadores serão descritos diretamente neste notebook para documentação.

A base inicial para os indicadores é o capítulo 7, Eficiência nas macrorregiões de saúde no Sistema Único de Saúde: uma abordagem comparativa – 2008-2017, do livro [SUS: avaliação da eficiência do gasto público em saúde](https://repositorio.ipea.gov.br/handle/11058/12029). Este capítulo busca associar os custos e infraestrutura relacionada ao SUS e o impacto na saúde da população de cada macrorregião, utilizando as seguintes variáveis:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Despesas per capita em atenção básica e vigilância;
- Despesas per capita em atenção especializada e Suporte profilático terapêutico;
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Equipamentos respiradores e ventiladores SUS/hab. (x100.000);
- População (em mil hab.);
- Área (km2);
- Densidade demográfica (hab./km²);
- Idosos %;
- PIB per capita;
- Usuários de planos de saúde %.


A partir dessas referência, o GT avaliou a disponibilidade de dados semelhantes a nível intramunicipal e complementou com informações de produção dos serviços de saúde, chegando na seguinte lista de variáveis, sempre que possível agregando por Subprefeitura:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Leitos de UTI/hab. (x100.000);
- Despesas liquidadas em atenção básica e vigilância;
- Despesas liquidadas em atenção especializada e Suporte profilático terapêutico;
- Consultas na Atenção Básica;
- Consultas na Atenção Especializada;
- Consultas na Urgência/Emergência;
- Número de Unidades Básicas de Saúde;
- Número de Ambulatórios especializados;
- Número de Unidades de Saúde Mental;
- Número de Unidades de Urgência e Emergência;
- Número de Hospitais;
- Número de Unidades DST/AIDS;
- Número de demais unidades;

A partir dessa definição, vamos obter as variáveis.

# Extração das variáveis

## Número de unidades

Estabelecimentos de saúde municipais, estaduais, federais e privados do município estão disponíveis no GeoSampa, em diferentes camadas. Primeiro, vamos avaliar quais são essas camadas e, depois, baixá-las e agrupá-las da maneira decidida pelo GT.

In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display, HTML


from core.downloads.geosampa import get_capabilities, get_features

In [2]:
equipamento_saude_typenames = get_capabilities('equipamento_saude')
equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_abrangencia_ubs',
  'title': 'Abrangência das Unidades Básicas de Saúde',
  'abstract': 'Áreas de abrangência das UBS.'},
 {'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_coordenadoria_regional',
  'title': 'Coordenadoria Regional de Saúde',
  'abstract': 'Limites administrativos da Coordenadoria Regional de Saúde.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que real

Primeiro, removemos todas as camadas que não representam os pontos das unidades de saúde. Além disso, a camada de vigilância não retorna nenhuma unidade, então vamos removê-la também.

In [3]:
del equipamento_saude_typenames[0]
del equipamento_saude_typenames[1]
del equipamento_saude_typenames[4]
del equipamento_saude_typenames[8]
del equipamento_saude_typenames[7]

equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que realizam procedimentos de média e alta complexidade.'},
 {'name': 'geoportal:equipamento_saude_outros',
  'title': 'Saúde - Outros',
  'abstract': 'Estabelecimentos e serviços especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo.'},
 {'name': 'geoportal:equipamento_saude_saude_mental

In [4]:
gdfs_equip = []

for t in equipamento_saude_typenames:
    gdfs_equip.append(get_features(t.get('name')))

In [6]:
for gdf in gdfs_equip:
    display(gdf.crs)
    display(gdf.head(2))

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,1,CAPAO REDONDO - AMA ESPECIALIDADES,"AV COMEN SANT'ANNA, 774",JD. BOA ESPERANCA,05866000\n,None,58729902,None,POINT (319108.42 7380764.47)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,2,PQ PERUCHE-AMA ESPECIALIDADES,"R JOSE RANGEL DE CAMARGO, 500",PQ PERUCHE,02538010\n,None,38589865,None,POINT (331055.43 7400384.23)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_hospital.120794,120794,6,SAUDE,0,SEM REDE,Sem descrição,30,Hospital,Realiza atendimento de média e alta complexidade.,...,Administração privada (inclui estabelecimentos...,281,SANTA MARIA CRUZ AZUL - HOSP,"AV LINS DE VASCONCELOS, 356",CAMBUCI,01538000\n,None,33484000,None,POINT (334481.53 7392822.24)
1,equipamento_saude_hospital.120795,120795,6,SAUDE,0,SEM REDE,Sem descrição,30,Hospital,Realiza atendimento de média e alta complexidade.,...,Administração privada (inclui estabelecimentos...,282,CASA DA CRIANCA BETINHO LAR ESPIRITA P/ EXCEPC...,"R VACANGA, 300",V CARRAO,03433020\n,None,67817366,None,POINT (343610.97 7393605.47)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

...

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ubs_posto_centro.120271,120271,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",...,Administração municipal,16,UBS JARDIM CLIPER,"R BELIZARIO FERREIRA LIMA,11",JD BELA VISTA,04829020\n,None,59285703,None,POINT (326077.574 7375311.115)
1,equipamento_saude_ubs_posto_centro.120272,120272,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",...,Administração municipal,206,JD SAO FRANCISCO II - AMA/UBS INTEGRADA,"R BANDEIRA DE ARACAMBI, 704",JD SAO FRANCISCO,08310010\n,None,27516712,None,POINT (351299.99 7385828.42)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_unidades_dst-aids.121019,121019,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,...,Administração municipal,219,PQ IPE- CTA DST/AIDS,"R FRANCISCO ANTUNES MEIRA, 255",PQ IPE,05742060\n,None,58428962,None,POINT (320361.91 7384500.83)
1,equipamento_saude_unidades_dst-aids.121020,121020,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,...,Administração municipal,220,CIDADE TIRADENTES-CTA DST/AIDS,"R LUIS BORDESE, 96",CIDADE TIRADENTES,08471780\n,None,22827055,None,POINT (356673.7 7389004.94)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_urgencia_emergencia.120738,120738,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,41,COMPLEXO PRATES - AMA,"R PRATES, 1101",BOM RETIRO,01009000\n,None,32298192,None,POINT (333142.32 7397479.26)
1,equipamento_saude_urgencia_emergencia.120739,120739,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,42,JD ELISA MARIA - AMA,"R ROMULO NALDI, 57",JD ELISA MARIA,02873250\n,None,39810967,None,POINT (327799.8 7404943.83)


In [8]:
gdf_equip = pd.concat(gdfs_equip)
gdf_equip = gpd.GeoDataFrame(gdf_equip)
gdf_equip = gdf_equip.set_geometry(gdfs_equip[0].geometry.name)
gdf_equip = gdf_equip.set_crs(gdfs_equip[0].crs)
gdf_equip

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,1,CAPAO REDONDO - AMA ESPECIALIDADES,"AV COMEN SANT'ANNA, 774",JD. BOA ESPERANCA,05866000\n,None,58729902,None,POINT (319108.42 7380764.47)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,2,PQ PERUCHE-AMA ESPECIALIDADES,"R JOSE RANGEL DE CAMARGO, 500",PQ PERUCHE,02538010\n,None,38589865,None,POINT (331055.43 7400384.23)
2,equipamento_saude_ambulatorios_especializados....,120003,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,3,V CONSTANCIA -AMA ESPECIALIDADES,"R HERMENEGILDO MARTINI, SN",V CONSTANCIA,04438280\n,None,55626944,None,POINT (330196.17 7380955.02)
3,equipamento_saude_ambulatorios_especializados....,120004,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,4,JD ICARAI I (QUINTANA) -AMA ESPECIALIDADES,"R S ROQUE DO PARAGUACU, 190",V QUINTANA,04837150\n,None,59280272,None,POINT (326183.88 7373067.87)
4,equipamento_saude_ambulatorios_especializados....,120005,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,5,ITAQUERA-AMA ESPECIALIDADES,"R AMERICO SALVADOR NOVELLI, 265",ITAQUERA,08210090\n,None,20712422,None,POINT (351434.05 7396009.19)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,equipamento_saude_urgencia_emergencia.120789,120789,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1169,UPA MOOCA III DOM PAULO EVARISTO ARNS,R DR FOMM 261,BELENZINHO,03163030\n,None,None,None,POINT (337224.82 7395152.06)
52,equipamento_saude_urgencia_emergencia.120790,120790,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1180,UPA CIDADE TIRADENTES,R IGARAPE DA DIANA 1,CONJ HAB INACIO MONTEIRO,08472170\n,None,None,None,POINT (356810.52 7392545.06)
53,equipamento_saude_urgencia_emergencia.120791,120791,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1181,UPA CITY JARAGUA,ESTRADA DAS TAIPAS 1648,JARAGUA,02991000\n,None,None,None,POINT (322821.39 7406568.93)
54,equipamento_saude_urgencia_emergencia.120792,120792,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1182,UPA JABAQUARA,R CRUZ DAS ALMAS 290,VILA CAMPESTRE,04330060\n,None,2476-7356,None,POINT (332239.79 7383268.35)
